In [23]:
import pandas as pd
from tqdm import tqdm
from scipy import stats
import numpy as np

In [25]:
def backtest_code(res, code, benchmark = 2, benchmark_index="US100"):
    rolling = 5
    pivoted = res[[code, benchmark_index]]
    pivoted.ffill()
    ret = pivoted.pct_change(fill_method=None)
    try:
        ret['stock_price'] = pivoted[code]
        ret['ret_diff'] = ret[code] - ret[benchmark_index]
        # type 1
        # ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std() * 100
        # ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean() * 100
        # ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])/ret['ret_diff_std'])
        
        # type 2
        ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std()
        ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean()
        ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])*ret['ret_diff_std'])*10000

        ret[code] = (1 + ret[code]).cumprod() - 1

        ret['index'] = [i for i in range(len(ret))]

        # check direction
        ret['lag10'] = ret['stock_price'].shift(-4)
        ret['change_stock'] = (ret['lag10'] - ret[code])/ret[code]
        ret['direction'] = ret.apply(lambda x: -1 if x['stock_price'] - x['lag10'] > 0 else 1, axis=1)

        # check if the direction are the same, if not, tned to not being a mean reversion
        ret['lag10_twii'] = ret[benchmark_index].shift(-4)
        ret['change_twii'] = (ret['lag10_twii'] - ret[benchmark_index])/ret[benchmark_index]
        ret['diff_lag'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 1, axis=1)

        # check raise
        ret[f'{code}_lag60'] = ret['stock_price'].shift(-12)
        ret['target_60mins'] = (ret[f'{code}_lag60'] - ret['stock_price'])/ret['stock_price'] * ret['direction']
        ret = ret[(ret['stdize_ret_diff'] >= benchmark) & ret['diff_lag'] == 1][['target_60mins', 'stdize_ret_diff']].dropna()
        return ret
    except:
        print(pivoted)

# US100

In [22]:
us_index = "US100"
res = pd.read_parquet(f"{us_index}.parquet")
code_list=list(set(res.columns))

dfs = []
for code in tqdm(code_list[:]):
    if code == us_index: continue
    df = backtest_code(res, code, benchmark=2, benchmark_index=us_index)
    if len(df) == 0: continue
    dfs.append(df)


# to test if return after 60 mins are greater than {test}
test = 0.01
final = pd.concat(dfs)
print(len(final))
final = final[final['target_60mins'] != 0.0]
final['target_60mins'] = final['target_60mins'] - test
print(len(final))
mean = final['target_60mins'].mean()
n = len(final)
s = final['target_60mins'].std()

t_statistic = (mean - 0.0) / (s / np.sqrt(n))
print(f"T-statistic: {t_statistic}")

t_stat, p_value = stats.ttest_1samp(final['target_60mins'], 0.0)

if t_stat > 0:
    one_tailed_p_value = p_value / 2
else:
    one_tailed_p_value = 1 - (p_value / 2)

print(f"P-value: {one_tailed_p_value}")

798
795
T-statistic: 0.5299965688433509
P-value: 0.29813124044082423


# US500

In [28]:
us_index = "US100"
res = pd.read_parquet(f"US500.parquet")
code_list=list(set(res.columns))

dfs = []
for code in tqdm(code_list[:]):
    if code == us_index: continue
    df = backtest_code(res, code, benchmark=2, benchmark_index=us_index)
    if len(df) == 0: continue
    dfs.append(df)

100%|██████████| 502/502 [00:15<00:00, 33.17it/s]


In [32]:
# to test if return after 60 mins are greater than {test}
test = 0.009
final = pd.concat(dfs)
print(len(final))
final = final[final['target_60mins'] != 0.0]
final['target_60mins'] = final['target_60mins'] - test
print(len(final))
mean = final['target_60mins'].mean()
n = len(final)
s = final['target_60mins'].std()

t_statistic = (mean - 0.0) / (s / np.sqrt(n))
print(f"T-statistic: {t_statistic}")

t_stat, p_value = stats.ttest_1samp(final['target_60mins'], 0.0)

if t_stat > 0:
    one_tailed_p_value = p_value / 2
else:
    one_tailed_p_value = 1 - (p_value / 2)

print(f"P-value: {one_tailed_p_value}")

2507
2500
T-statistic: 4.093363030503
P-value: 2.193255260061279e-05
